In [1]:
!nvidia-smi

Wed Nov 20 04:42:41 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   56C    P8              13W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install datasets accelerate peft bitsandbytes transformers trl
!pip install safetensors
!pip install wandb
!export WANDB_API_KEY='8a0bddf7bc0eea5f94769fd8debe3af8a3aa7a9e'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 143.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 115.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 32.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
   

In [3]:
import os
import wandb

api_key = os.getenv('WANDB_API_KEY')
wandb.login(key=api_key)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
wandb.init(
    project="ddos-detection",
    name="llm-evaluation-run",
    config={
        "model": "fine-tuned-llm",
        "dataset": "BCCC-Cpacket-Cloud-DDoS-2024"
    }
)

wandb: Currently logged in as: kyousaf (kyousaf-carnegie-mellon-university). Use `wandb login --relogin` to force relogin


In [5]:
import os
import torch
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer
import pandas as pd

In [6]:
df_final = pd.read_csv('llama-2-cpacket-data.csv')

In [7]:
df_final.head()

,text
0,<s>[INST] The network flow has total header by...
1,<s>[INST] The network flow has total header by...
2,<s>[INST] The network flow has total header by...
3,<s>[INST] The network flow has total header by...
4,<s>[INST] The network flow has total header by...


In [8]:
dataset = Dataset.from_pandas(df_final)
dataset = dataset.shuffle(seed=42)

train_test_split = dataset.train_test_split(test_size=0.2, seed=42)  # 80% train, 20% validation
train_dataset = train_test_split['train']
val_dataset = train_test_split['test']

In [9]:
# Import necessary libraries
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import PeftModel
from datasets import load_dataset

In [10]:
# Load your validation dataset
#val_dataset = load_dataset('your_val_dataset')  # Replace with your actual dataset

# Preprocess the val_dataset for Post-Training Evaluation

# Function to split the prompt and expected response
def preprocess_data(example):
    prompt = example['text'].split("[/INST]")[0] + "[/INST]"  # Prompt
    expected_response = example['text'].split("[/INST]")[1].strip()  # Expected response

    return {
        'prompt': prompt,
        'expected_response': expected_response
    }

In [11]:
# Apply preprocessing to your validation dataset
val_dataset = val_dataset.map(preprocess_data)

Map:   0%|          | 0/953 [00:00<?, ? examples/s]

In [12]:
# Set compute dtype
compute_dtype = getattr(torch, "float16")

# Quantization configuration (same as during training)
from transformers import BitsAndBytesConfig
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [13]:
print(torch.cuda.is_available())  # This should return True if GPU is available
print(torch.cuda.device_count())

True
1


In [14]:
base_model = "NousResearch/Llama-2-7b-chat-hf"

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0},
    torch_dtype=compute_dtype
)

# Disable cache if necessary
model.config.use_cache = False

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [15]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [63]:
# Load the base model with quantization
from peft import PeftModel
from safetensors.torch import load_file

# Load the LoRA adapter weights (your fine-tuned model)
# adapter_path = '/content/results/checkpoint-3600'  # Replace with the path to your adapter files
adapter_path = '/content/model/2-epoch-best-run/'  # Replace with the path to your adapter files

model = PeftModel.from_pretrained(
    model,
    adapter_path,
    local_files_only=True  # Force loading from local files
)

In [64]:
# Initialize the text generation pipeline with your fine-tuned model
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=500,
)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausa

In [68]:
val_dataset

Dataset({
    features: ['text', 'prompt', 'expected_response'],
    num_rows: 953
})

In [66]:
!pip install tqdm
import tqdm

In [77]:
val_dataset

Dataset({
    features: ['text', 'prompt', 'expected_response'],
    num_rows: 953
})

In [94]:
df_final.head(1)

,text
0,<s>[INST] The network flow has total header by...


In [98]:
# prompt: show me 2 full rows of df_final without ...

print(df_final.head(2).to_string(index=False))

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          text
<s>[INST] The network flow has total header bytes of 40 and max header bytes of 40 and min header bytes of 40. The mean header bytes are 40.00 while the mode is 40.0. The forward total header bytes are 40 and forward mode header bytes are 40.0. The forward initial window bytes are 64240 and backward initial window bytes are 0. 

In [99]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from tqdm import tqdm

# Set up device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model.eval()  # Set model to evaluation mode

# Adjust batch size based on GPU memory
batch_size = 32  # Increase if your GPU has more memory

generated_labels = []

# Prepare prompts from the validation dataset
prompts = [example['prompt'] for example in val_dataset]

# Process prompts in batches
for i in tqdm(range(0, len(prompts), batch_size)):
    batch_prompts = prompts[i:i+batch_size]

    # Tokenize the batch prompts
    inputs = tokenizer(
        batch_prompts,
        return_tensors='pt',
        padding=True,
        truncation=True,
        max_length=512  # Adjust if needed
    ).to(device)

    # Generate outputs
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=50,  # Adjust if needed
            do_sample=False,    # Faster inference with greedy decoding
            num_beams=1         # Set to 1 for faster inference
        )

    # Decode outputs
    generated_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    for generated_text in generated_texts:
        # Extract the generated response (remove the prompt part)
        if "[/INST]" in generated_text:
            generated_response = generated_text.split("[/INST]")[1].strip()
        else:
            generated_response = generated_text.strip()

        # Determine the label from the response
        if "Attack" in generated_response:
            generated_labels.append("Attack")
        elif "Benign" in generated_response:
            generated_labels.append("Benign")
        elif "Suspicious" in generated_response:
            generated_labels.append("Suspicious")
        else:
            generated_labels.append("Unknown")  # Handle unexpected cases


  0%|          | 0/30 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
100%|██████████| 30/30 [05:59<00:00, 11.98s/it]


In [103]:
# Extract true labels using list comprehension
true_labels = [
    "Attack" if "Attack" in example['expected_response'] else
    "Benign" if "Benign" in example['expected_response'] else
    "Suspicious" if "Suspicious" in example['expected_response'] else
    "Unknown"
    for example in val_dataset
]


In [105]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Compute accuracy
accuracy = accuracy_score(true_labels, generated_labels)

# Generate classification report
report = classification_report(
    true_labels,
    generated_labels,
    labels=["Attack", "Benign", "Suspicious", "Unknown"],
    output_dict=True
)

# Compute confusion matrix
conf_matrix = confusion_matrix(
    true_labels,
    generated_labels,
    labels=["Attack", "Benign", "Suspicious", "Unknown"]
)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [107]:
import matplotlib.pyplot as plt
import seaborn as sns

# Log accuracy
wandb.log({'Accuracy': accuracy})

# Log classification metrics for each class
class_labels = ["Attack", "Benign", "Suspicious", "Unknown"]
for label in class_labels:
    wandb.log({
        f"{label} Precision": report[label]['precision'],
        f"{label} Recall": report[label]['recall'],
        f"{label} F1-Score": report[label]['f1-score'],
        f"{label} Support": report[label]['support'],
    })

# Log macro and weighted averages
wandb.log({
    'Macro Avg Precision': report['macro avg']['precision'],
    'Macro Avg Recall': report['macro avg']['recall'],
    'Macro Avg F1-Score': report['macro avg']['f1-score'],
    'Weighted Avg Precision': report['weighted avg']['precision'],
    'Weighted Avg Recall': report['weighted avg']['recall'],
    'Weighted Avg F1-Score': report['weighted avg']['f1-score'],
})

# Plot and log confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(
    conf_matrix,
    annot=True,
    fmt='d',
    cmap='Blues',
    xticklabels=class_labels,
    yticklabels=class_labels
)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')

# Save the confusion matrix plot
plt.savefig('confusion_matrix.png')
plt.close()

# Log the confusion matrix image to wandb
wandb.log({"Confusion Matrix": wandb.Image('confusion_matrix.png')})



In [111]:
#print accuracy, report and conf_matrix
print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.6547743966421826
Classification Report:
{'Attack': {'precision': 0.7547683923705722, 'recall': 0.8393939393939394, 'f1-score': 0.7948350071736011, 'support': 330.0}, 'Benign': {'precision': 0.9415584415584416, 'recall': 0.4559748427672956, 'f1-score': 0.614406779661017, 'support': 318.0}, 'Suspicious': {'precision': 0.6941580756013745, 'recall': 0.6622950819672131, 'f1-score': 0.6778523489932886, 'support': 305.0}, 'Unknown': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 0.0}, 'accuracy': 0.6547743966421826, 'macro avg': {'precision': 0.5976212273825972, 'recall': 0.489415966032112, 'f1-score': 0.5217735339569767, 'support': 953.0}, 'weighted avg': {'precision': 0.7976992308040844, 'recall': 0.6547743966421826, 'f1-score': 0.6971897951127438, 'support': 953.0}}
Confusion Matrix:
[[277   5  40   8]
 [  2 145  49 122]
 [ 88   4 202  11]
 [  0   0   0   0]]


In [112]:
# Compare the extracted labels and compute accuracy
from sklearn.metrics import accuracy_score, classification_report

# Calculate accuracy
accuracy = accuracy_score(true_labels, generated_labels)
print(f"Accuracy: {accuracy:.6f}")

# Optionally, print a more detailed classification report
print(classification_report(true_labels, generated_labels, labels=["Attack", "Benign", "Suspicious"]))

Accuracy: 0.654774
              precision    recall  f1-score   support

      Attack       0.75      0.84      0.79       330
      Benign       0.94      0.46      0.61       318
  Suspicious       0.69      0.66      0.68       305

   micro avg       0.77      0.65      0.71       953
   macro avg       0.80      0.65      0.70       953
weighted avg       0.80      0.65      0.70       953



In [114]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import wandb
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from tqdm import tqdm

# Set up device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load tokenizer and model (replace 'your-model-name' with your actual model name)
model.eval()

# Adjust batch size based on GPU memory
batch_size = 32  # Increase if your GPU has more memory

# Prepare prompts and expected responses from the validation dataset
prompts = [example['prompt'] for example in val_dataset]
expected_responses = [example['expected_response'] for example in val_dataset]

# Extract true labels using list comprehension
true_labels = [
    "Attack" if "Attack" in expected_response else
    "Benign" if "Benign" in expected_response else
    "Suspicious" if "Suspicious" in expected_response else
    "Unknown"
    for expected_response in expected_responses
]

# Initialize lists to store results
generated_labels = []
generated_responses = []
data_records = []

# Process prompts in batches
for i in tqdm(range(0, len(prompts), batch_size)):
    batch_prompts = prompts[i:i+batch_size]
    batch_expected_responses = expected_responses[i:i+batch_size]
    batch_true_labels = true_labels[i:i+batch_size]

    # Tokenize the batch prompts
    inputs = tokenizer(
        batch_prompts,
        return_tensors='pt',
        padding=True,
        truncation=True,
        max_length=512  # Adjust if needed
    ).to(device)

    # Generate outputs
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=50,  # Adjust if needed
            do_sample=False,    # Faster inference with greedy decoding
            num_beams=1         # Set to 1 for faster inference
        )

    # Decode outputs
    generated_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    # Process each item in the batch
    for j in range(len(batch_prompts)):
        prompt = batch_prompts[j]
        expected_response = batch_expected_responses[j]
        true_label = batch_true_labels[j]
        generated_text = generated_texts[j]

        # Extract the generated response (remove the prompt part)
        if "[/INST]" in generated_text:
            generated_response = generated_text.split("[/INST]", 1)[1].strip()
        else:
            generated_response = generated_text.strip()

        # Determine the label from the generated response
        if "Attack" in generated_response:
            generated_label = "Attack"
        elif "Benign" in generated_response:
            generated_label = "Benign"
        elif "Suspicious" in generated_response:
            generated_label = "Suspicious"
        else:
            generated_label = "Unknown"  # Handle unexpected cases

        # Append results to lists
        generated_labels.append(generated_label)
        generated_responses.append(generated_response)

        # Collect data for CSV
        data_records.append({
            'Prompt': prompt,
            'Expected Response': expected_response,
            'Generated Response': generated_response,
            'True Label': true_label,
            'Generated Label': generated_label
        })

# Create a DataFrame and save to CSV
df_results = pd.DataFrame(data_records)
df_results.to_csv('evaluation_results.csv', index=False)


  0%|          | 0/30 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
100%|██████████| 30/30 [05:59<00:00, 11.98s/it]


In [119]:
# prompt: show first 1full  row of column Generated Response in df_results

print(df_results['Generated Response'].iloc[0])

This data flow can be categorized as Benign. The header bytes are all 24, which is the expected value, and the forward initial window bytes are 65535, which is the maximum allowed value. The inter-
